In [8]:
import pandas as pd
import os
import random
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [32]:
data_dir = '/opt/ml/input/data/'
csv_file_path = os.path.join(data_dir, 'train_data_user_fe.csv')
df = pd.read_csv(csv_file_path)
df.head(2).T

,0,1
userID,0,0
assessmentItemID,A060001001,A060001002
testId,A060000001,A060000001
answerCode,1,1
Timestamp,1585009031,1585009034
KnowledgeTag,7224,7225
year,2020,2020
month,3,3
day,24,24
hour,0,0


In [15]:
def item_fe(df):
    df['item_ans_1'] = df[df['userID'].shift(1) == df['userID']].groupby('assessmentItemID')['answerCode'].cumsum()
    df['item_total_ans'] = df.groupby('assessmentItemID')['answerCode'].cumcount() + 1
    df['item_acc'] = df['item_ans_1']/df['item_total_ans']

    
    ## item의 평균 정답률, 정답 총합, 표준편차
    correct_a = df.groupby(['assessmentItemID'])['answerCode'].agg(['mean', 'sum','std'])
    correct_a.columns = ["item_ans_mean", 'item_ans_sum','item_ans_std']
    df = pd.merge(df, correct_a, on=['assessmentItemID'], how="left")
    
    ## item의 평균 풀이 시간, 표준편차
    time_a = df.groupby(['assessmentItemID'])['time'].agg(['mean', 'std'])
    time_a.columns = ["item_time_mean", 'item_time_std']
    df = pd.merge(df, time_a, on=['assessmentItemID'], how="left")

    ## item 난이도
    df['item_lvl'] = df['item_time_mean'] / df['item_ans_mean']
    # 범주화
    item_cat_num = 10
    df['item_lvl_cat'] = pd.qcut(df['item_lvl'],item_cat_num,labels=[i for i in range(item_cat_num)])

    ## item 노출 횟수
    df['item_total_answer'] = df.groupby('assessmentItemID')['answerCode'].cumcount()
    
    df = df.fillna(0)
    return df

In [17]:
df = item_fe(df)

In [31]:
df[df['userID'].shift(1) != df['userID']]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,year,month,day,hour,...,user_acc,user_test_ans_count,user_test_ans_1,user_test_acc,user_test_lvl_mean,user_tag_ans_count,user_tag_ans_1,user_tag_acc,user_tag_lvl_mean,user_lvl
0,0,A060001001,A060000001,1,1585009031,7224,2020,3,24,0,...,NaN,0,NaN,NaN,23.877991,0,NaN,NaN,14.685746,38.563737
745,1,A040013001,A040000013,1,1578300043,2048,2020,1,6,8,...,NaN,0,NaN,NaN,93.039942,0,NaN,NaN,107.943000,200.982943
1678,2,A030050001,A030000050,1,1578654173,407,2020,1,10,11,...,NaN,0,NaN,NaN,61.072497,0,NaN,NaN,57.477891,118.550387
1954,5,A080001001,A080000001,1,1581537620,4604,2020,2,12,20,...,NaN,0,NaN,NaN,42.508562,0,NaN,NaN,57.005571,99.514133
2787,6,A030016001,A030000016,1,1583750908,7307,2020,3,9,10,...,NaN,0,NaN,NaN,81.979383,0,NaN,NaN,51.957269,133.936653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037164,1556,A040007001,A040000007,1,1579520129,7271,2020,1,20,11,...,NaN,0,NaN,NaN,33.893922,0,NaN,NaN,34.131471,68.025393
1037897,1560,A080002001,A080000002,1,1582852455,4605,2020,2,28,1,...,NaN,0,NaN,NaN,89.469776,0,NaN,NaN,60.694836,150.164612
1038184,1561,A020005001,A020000005,0,1583120441,7912,2020,3,2,3,...,NaN,0,NaN,NaN,49.386935,0,NaN,NaN,54.635967,104.022903
1038677,1562,A070002001,A070000002,1,1582004070,607,2020,2,18,5,...,NaN,0,NaN,NaN,110.788896,0,NaN,NaN,85.162461,195.951356


In [18]:
df['item_acc']

0          1.000000
1          1.000000
2          1.000000
3          1.000000
4          1.000000
             ...   
2266581    0.435252
2266582    0.649635
2266583    0.635036
2266584    0.791971
2266585    0.543796
Name: item_acc, Length: 2266586, dtype: float64